#CafChem tools for using PySCF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/PySCF_CafChem.ipynb)

## This notebook allows you to:
- Install and load the PySCF, Geometric and RDKit libraries
- Runn the following PySCF calculations:
  * Hartree Fock
  * DFT
  * TDDFT
  * Implicit solvent
  * Geometry optimizations
  * MP2
  * CCSD(T)
  *Molecular Dynamics

## Requirements:
- This notebook will run faster with more CPU cores (High memoery runtimes have more cores: 8 compared to 2 with the normal CPU runtime)
- GPU-based PySCF does exist, but not here

## Acknowledgements:
- [see the documentation for more details](https://pyscf.org/user/index.html)

## Install and import

In [1]:
!pip install -q pyscf geometric rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 65.5 MB/s eta 0:00:00


In [36]:
from IPython.utils.sysinfo import num_cpus
from pyscf import scf, dft, gto, mp, tddft, cc
import pyscf
import pyscf.md
from pyscf.geomopt.geometric_solver import optimize
from pyscf import lib
from rdkit import Chem
from rdkit.Chem import AllChem
import time, datetime

#get number of cores
num_cores = num_cpus()
print(f'Number of cores: {num_cores}')
lib.num_threads(num_cores)

Number of cores: 8


8

## Function for smiles -> PySCF Mol object

In [3]:
def smiles_to_mol(smiles: str, basis: str = 'sto3g', charge: int = 0, spin:int = 0):
  '''
  receives a smiles string and returns a pyscf mole object. Adds Hs to
  molecule, optimizes with MMFF by RDKit. Makes and XYZ string,
  and a Mole object.

    Args:
      smiles: SMILES string for molecule
      charge: charge of molecule
      spin: spin multiplicity of molecule

    Returns:
      new_mol: pyscf Mole object
  '''
  atoms_list = ""
  mol = Chem.MolFromSmiles(smiles)
  molH = Chem.AddHs(mol)
  AllChem.EmbedMolecule(molH)
  AllChem.MMFFOptimizeMolecule(molH)
  xyz_string = f""
  for atom in molH.GetAtoms():
    atoms_list += atom.GetSymbol()
    pos = molH.GetConformer().GetAtomPosition(atom.GetIdx())
    xyz_string += f"{atom.GetSymbol()} {pos[0]} {pos[1]} {pos[2]}; "

  new_mol = gto.M(
    atom  = xyz_string,
    charge = charge,
    spin = spin,
    basis = basis,
    symmetry= True
)

  return new_mol

## Basis sets:

| Pople | Dunning | Karlsruhe | Pseudopotentials | Other |
| -- | -- | -- | -- | -- |
 | sto3g  | cc-pvdz   | def2-svp   | gth-szv   | minao
 |3-21g  | cc-pvtz   | def2-tzvp   | gth-dzv    | faegre_dz
 | 6-31g  | cc-pvqz   | def2-qzvp   | gth-dzvp
 | 6-311g  | aug-cc-pvdz   | gth-tzvp
 | 6-311++g**  | etc
 | etc  |



## HF

In [12]:
# set up a mol object manually

lih_mol = gto.M(
    atom  = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.6',
    basis = 'def2-tzvp',
    symmetry= True
)

In [27]:
test_mol = smiles_to_mol('CC=O')

scf_test = scf.RHF(test_mol)

start = time.time()
energy = scf_test.kernel()
end = time.time()

print(f'The HF energy is: {energy}')
print(f'The HF calculation took {end - start} seconds')

converged SCF energy = -150.94440780447
The HF energy is: -150.94440780447044
The HF calculation took 0.6825015544891357 seconds


## DFT

In [28]:
functional = 'b3lyp' # 'wB97m_v'

test_mol = smiles_to_mol('CC=O')

dft_test = dft.RKS(test_mol)
dft_test.xc = functional

start = time.time()
energy = dft_test.kernel()
end = time.time()

print(f'The DFT energy with {functional} is: {energy}')
print(f'The DFT calculation took {end - start} seconds')

converged SCF energy = -151.813317224586
The DFT energy with b3lyp is: -151.8133172245858
The DFT calculation took 3.279045343399048 seconds


### DFT Solvation

In [29]:
functional = 'b3lyp' # 'wB97m_v'

test_mol = smiles_to_mol('CC=O')
dft_test = dft.RKS(test_mol)
dft_test.xc = functional

solv_test = dft_test.PCM()
solv_test.with_solvent.method = 'IEF-PCM' # C-PCM, SS(V)PE, COSMO
#solv_test.with_solvent.eps = 32.613   # methanol (water is default)

start = time.time()
energy = solv_test.kernel()
end = time.time()

print(f'The solvatd DFT energy with {functional} is: {energy}')
print(f'The DFT calculation took {end - start} seconds')


WARN: PCM is an experimental feature. It is still in testing.
Features and APIs may be changed in the future.

converged SCF energy = -151.815914064659
The solvatd DFT energy with b3lyp is: -151.81591406465878
The DFT calculation took 5.418629169464111 seconds


### TDDFT

In [30]:
functional = 'b3lyp' # 'wB97m_v'

test_mol = smiles_to_mol('CC=O')

dft_test = dft.RKS(test_mol)
dft_test.xc = functional

start = time.time()
energy = dft_test.kernel()
end = time.time()

print(f'The DFT energy with {functional} is: {energy}')
print(f'The DFT calculation took {end - start} seconds')

test_td = tddft.TDDFT(dft_test)
test_td.nstates = 10
start = time.time()
test_td.kernel()
end = time.time()

test_td.analyze()

print(f'The TDDFT calculation took {end - start} seconds')

converged SCF energy = -151.813317041408
The DFT energy with b3lyp is: -151.81331704140797
The DFT calculation took 2.9050815105438232 seconds
Excited State energies (eV)
[ 3.98314708  9.0688712  10.64304338 10.74929469 12.13729724 12.59946584
 13.79019994 14.16189051 15.21983739 15.63706599]

** Singlet excitation energies and oscillator strengths **
Excited State   1:    A      3.98315 eV    311.27 nm  f=0.0000
Excited State   2:    A      9.06887 eV    136.71 nm  f=0.0027
Excited State   3:    A     10.64304 eV    116.49 nm  f=0.0026
Excited State   4:    A     10.74929 eV    115.34 nm  f=0.1831
Excited State   5:    A     12.13730 eV    102.15 nm  f=0.0189
Excited State   6:    A     12.59947 eV     98.40 nm  f=0.0075
Excited State   7:    A     13.79020 eV     89.91 nm  f=0.1298
Excited State   8:    A     14.16189 eV     87.55 nm  f=0.0687
Excited State   9:    A     15.21984 eV     81.46 nm  f=0.0019
Excited State  10:    A     15.63707 eV     79.29 nm  f=0.0146
The TDDFT calcul

### DFT Geometry optimization

In [31]:
functional = 'b3lyp' # 'wB97m_v'

test_mol = smiles_to_mol('CC=O')

opt_test = dft.RKS(test_mol)
opt_test.xc = functional

start = time.time()
opt_final = opt_test.Gradients().optimizer(solver='geometric').kernel()
end = time.time()

print(f'The optimized DFT energy with {functional} is: {opt_final}')
print(f'The optimized DFT calculation took {end - start} seconds')
print(opt_final.tostring())

geometric-optimize called with the following command line:
/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-be297a66-2a55-4e8f-bb47-e7306990ec04.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.644187  -0.080845  -0.044531    0.000000  0.000000  0.000000
   C   0.838788   0.099298   0.060592   -0.000000  0.000000 -0.000000
   O   1.424768   1.111333  -0.309282    0.000000  0.000000  0.000000
   H  -1.056825  -0.239077   0.954530    0.000000  0.000000  0.000000
   H  -0.855175  -0.950151  -0.671603    0.000000  0.000000  0.000000
   H  -1.099224   0.807512  -0.489417    0.000000  0.000000  0.000000
   H   1.391856  -0.748070   0.499710    0.000000  0.000000  0.000000

WARN: Mole.unit (angstrom) is changed to Bohr

converged SCF energy = -151.813317601403
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C     0.0251856283     0.0036883560     0.0015110924
1 C    -0.0086924877     0.0138841336    -0.0071443169
2 O    -0.0268475748    -0.0419662101     0.0150034251
3 H     0.0066382074     0.00

Step    0 : Gradient = 2.471e-02/5.203e-02 (rms/max) Energy = -151.8133176014
Hessian Eigenvalues: 2.30000e-02 2.69327e-02 5.00000e-02 ... 3.45319e-01 3.45320e-01 9.25446e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.670155  -0.086220  -0.045372   -0.025967 -0.005375 -0.000841
   C   0.874605   0.153495   0.041550    0.035818  0.054197 -0.019043
   O   1.573526   1.139088  -0.302403    0.148758  0.027755  0.006878
   H  -1.094520  -0.268136   0.958262   -0.037695 -0.029059  0.003732
   H  -0.893627  -0.976033  -0.660802   -0.038452 -0.025882  0.010802
   H  -1.171428   0.787602  -0.489622   -0.072204 -0.019911 -0.000205
   H   1.384242  -0.749483   0.499523   -0.007615 -0.001414 -0.000187
converged SCF energy = -151.815808116822
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C     0.0016365896    -0.0123748105     0.0056138244
1 C    -0.0147195473     0.0133933913    -0.0076845852
2 O     0.0177622560     0.0016147414     0.0014614262
3 H    -0.0039285430    -0.0010429153     0.0001189289
4 H    -0.0039007785  

Step    1 : Displace = 7.411e-02/1.508e-01 (rms/max) Trust = 1.000e-01 (=) Grad = 1.197e-02/2.133e-02 (rms/max) E (change) = -151.8158081168 (-2.491e-03) Quality = 0.411
Hessian Eigenvalues: 2.30000e-02 2.69327e-02 4.99995e-02 ... 3.45319e-01 4.40746e-01 9.54484e-01


...


Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.664460  -0.079618  -0.047408    0.000565 -0.000629  0.000407
   C   0.877941   0.118065   0.058286   -0.003006  0.000455 -0.000210
   O   1.492398   1.142381  -0.312088    0.001512 -0.002537  0.002099
   H  -1.085998  -0.255338   0.956214    0.001730  0.000627  0.000887
   H  -0.883952  -0.964809  -0.667150    0.002220  0.000050 -0.000777
   H  -1.140787   0.805554  -0.493523    0.002472  0.000015  0.000370
   H   1.407555  -0.768714   0.511243   -0.002729  0.001259 -0.000720
converged SCF energy = -151.817847622585
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C    -0.0000744634     0.0000921697    -0.0000474275
1 C     0.0002184906     0.0002584516    -0.0000913898
2 O    -0.0002525370    -0.0003430463     0.0000986751
3 H     0.0000522688    -0.0000067582     0.0000423884
4 H     0.0000536830  

Step    4 : Displace = 2.495e-03/3.633e-03 (rms/max) Trust = 5.431e-02 (+) Grad = 2.238e-04/4.373e-04 (rms/max) E (change) = -151.8178476226 (-1.471e-05) Quality = 0.934
Hessian Eigenvalues: 2.29928e-02 2.69327e-02 4.94948e-02 ... 3.45612e-01 6.25581e-01 9.42877e-01



Geometry optimization cycle 6
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C  -0.664114  -0.079638  -0.047294    0.000346 -0.000020  0.000113
   C   0.878406   0.117597   0.058910    0.000465 -0.000468  0.000624
   O   1.493540   1.141961  -0.310999    0.001143 -0.000420  0.001090
   H  -1.086436  -0.255268   0.955982   -0.000438  0.000069 -0.000232
   H  -0.884123  -0.964522  -0.667237   -0.000171  0.000287 -0.000087
   H  -1.140080   0.805773  -0.493461    0.000706  0.000219  0.000062
   H   1.407793  -0.769520   0.511789    0.000238 -0.000807  0.000546
converged SCF energy = -151.81784778606
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 C    -0.0000390027    -0.0000164568     0.0000029985
1 C     0.0000261528     0.0000777256    -0.0000339030
2 O    -0.0000220984    -0.0000369751    -0.0000066800
3 H     0.0000093513    -0.0000036219    -0.0000040797
4 H     0.0000020903   

Step    5 : Displace = 2.970e-04/3.875e-04 (rms/max) Trust = 7.681e-02 (+) Grad = 4.120e-05/8.874e-05 (rms/max) E (change) = -151.8178477861 (-1.635e-07) Quality = 0.642
Hessian Eigenvalues: 2.29928e-02 2.69327e-02 4.94948e-02 ... 3.45612e-01 6.25581e-01 9.42877e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 12.748 seconds


The optimized DFT energy with b3lyp is: <pyscf.gto.mole.Mole object at 0x7be327100710>
The optimized DFT calculation took 12.780466794967651 seconds
C          -0.66411419       -0.07963762       -0.04729406
C           0.87840626        0.11759657        0.05891014
O           1.49354048        1.14196092       -0.31099881
H          -1.08643599       -0.25526824        0.95598222
H          -0.88412291       -0.96452212       -0.66723687
H          -1.14008029        0.80577265       -0.49346061
H           1.40779288       -0.76952032        0.51178900


## MP2

In [32]:
test_mol = smiles_to_mol('CC=O')

mp2_test = test_mol.RHF().run()

print(f'The HF energy is: {mp2_test.energy_tot()}')

start = time.time()
mp2_energy = mp2_test.MP2().run()
end = time.time()

print(f'The MP2 energy is: {mp2_energy.emp2 + mp2_test.energy_tot()}')
print(f'The MP2 calculation took {end - start} seconds')

converged SCF energy = -150.944407778187
The HF energy is: -150.94440777818733
E(RMP2) = -151.112957402031  E_corr = -0.168549623843994
E(SCS-RMP2) = -151.123230921434  E_corr = -0.178823143246817
The MP2 energy is: -151.11295740203133
The MP2 calculation took 0.0035414695739746094 seconds


### Alternate MP2

In [33]:
test_mol = smiles_to_mol('CC=O')

mp2_test = scf.HF(test_mol).run()

start = time.time()
final_mp2 = mp.MP2(mp2_test).run()
end = time.time()

print(f'The MP2 energy is: {final_mp2.e_tot}')
print(f'The MP2 calculation took {end - start} seconds')

converged SCF energy = -150.944407776981
E(RMP2) = -151.112957414759  E_corr = -0.168549637778017
E(SCS-RMP2) = -151.123230934173  E_corr = -0.178823157191883
The MP2 energy is: -151.1129574147592
The MP2 calculation took 0.004670381546020508 seconds


## CCSD(T)

In [34]:
test_mol = smiles_to_mol('CC=O')

hf_test = test_mol.RHF().run()

print(f'The HF energy is: {hf_test.energy_tot()}')

cc_test = cc.CCSD(hf_test)

start = time.time()
cc_test.kernel()
end = time.time()

# mycc = cc.CCSD(mf).run()

ccsd_energy = cc_test.e_tot
cc_correction = cc_test.energy()
pert = cc_test.ccsd_t()

print(f'The CCSD correction is {cc_correction}')
print(f'CCSD(T) total energy: {ccsd_energy + pert}')
print(f'CCSD(T) calculation took {end - start} seconds')

converged SCF energy = -150.944407870372
The HF energy is: -150.94440787037155
E(CCSD) = -151.1527509562277  E_corr = -0.2083430858561613
CCSD(T) correction = -0.00269670380459584
The CCSD correction is -0.20834308585616138
CCSD(T) total energy: -151.1554476600323
CCSD(T) calculation took 0.3004739284515381 seconds


## Dynamics  - Born Oppenheimer MD NVE

In [37]:
functional = 'b3lyp' # 'wB97m_v'

test_mol = smiles_to_mol('CC=O')

md_test = dft.RKS(test_mol)
md_test.xc = functional

start = time.time()
md_test.kernel()
end = time.time()

print(f'The DFT energy with {functional} is: {md_test.e_tot}')
print(f'The DFT calculation took {end - start} seconds')

myscanner = md_test.nuc_grad_method().as_scanner()

# sets the time step to 5 a.u. and will run for 100 steps
# or for 50 a.u.
myintegrator = pyscf.md.NVE(myscanner,
                            dt=5,
                            steps=100,
                            data_output="BOMD.md.data",
                            trajectory_output="BOMD.md.xyz").run()

myintegrator.data_output.close()
myintegrator.trajectory_output.close()

converged SCF energy = -151.813316211322
The DFT energy with b3lyp is: -151.8133162113217
The DFT calculation took 2.1919806003570557 seconds
data output file: BOMD.md.data
trajectory output file: BOMD.md.xyz

BOMD Time 0.00
converged SCF energy = -151.813316211623
--------------- SymAdaptedRKS_Scanner gradients ---------------
         x                y                z
0 C     0.0254872226    -0.0003762025     0.0007680900
1 C    -0.0071648548    -0.0161576702    -0.0026754151
2 O    -0.0312636764     0.0413095587     0.0052352238
3 H     0.0068716758    -0.0044134462    -0.0046483966
4 H     0.0030910030     0.0045338341     0.0007856958
5 H     0.0066059179    -0.0056550913     0.0035619962
6 H    -0.0036595251    -0.0191982319    -0.0030217724
----------------------------------------------
----------- VelocityVerlet final geometry -----------
         x                y                z
0 C    -1.2290447020    -0.0092617256    -0.0411830235
1 C     1.5991074532     0.0241774905  

## BOMD.md.data:

time          Epot                 Ekin                 Etot                 T


    0.00  -1.518133162116E+02  0.000000000000E+00  -1.518133162116E+02 0.0000

    5.00  -1.518133219933E+02  5.778556227819E-06  -1.518133162148E+02 0.1738

    10.00  -1.518133392601E+02  2.303604309575E-05  -1.518133162241E+02 0.6928

    15.00  -1.518133677788E+02  5.153964742868E-05  -1.518133162391E+02 1.5500

    20.00  -1.518134071665E+02  9.090715709262E-05  -1.518133162594E+02 2.7339

    25.00  -1.518134568996E+02  1.406155875675E-04  -1.518133162840E+02 4.2288

    30.00  -1.518135163249E+02  2.000128967075E-04  -1.518133163120E+02 6.0152

    35.00  -1.518135846747E+02  2.683324365944E-04  -1.518133163423E+02 8.0698

    40.00  -1.518136610835E+02  3.447097185354E-04  -1.518133163737E+02 10.3667

    45.00  -1.518137446060E+02  4.282010085136E-04  -1.518133164050E+02 12.8776

    50.00  -1.518138342380E+02  5.178032297540E-04  -1.518133164348E+02 15.5723

    55.00  -1.518139289365E+02  6.124746274895E-04  -1.518133164618E+02 18.4195

    60.00  -1.518140276404E+02  7.111556483966E-04  -1.518133164848E+02 21.3872

    65.00  -1.518141292919E+02  8.127895026203E-04  -1.518133165024E+02 24.4437

    70.00  -1.518142328555E+02  9.163419081280E-04  -1.518133165136E+02 27.5579

    [...]

BOMD.md.xyz:

7

MD Time 0.00

C          -0.65038245       -0.00490109       -0.02179312

C           0.84621122        0.01279418        0.02932545

O           1.53898391       -0.99105787       -0.10006165

H          -1.04492334        0.36919744        0.92573889

H          -1.00788573       -1.02416111       -0.18750549

H          -0.98778550        0.63615090       -0.83949775

H           1.30578189        1.00197756        0.19379367

7

MD Time 5.00

C          -0.65039015       -0.00490098       -0.02179335

C           0.84621339        0.01279906        0.02932626

O           1.53899100       -0.99106724       -0.10006284

H          -1.04494808        0.36921333        0.92575563

H          -1.00789686       -1.02417743       -0.18750832

H          -0.98780928        0.63617126       -0.83951057

H           1.30579506        1.00204668        0.19380455

7

MD Time 10.00

C          -0.65041328       -0.00490063       -0.02179405

C           0.84621990        0.01281373        0.02932869

O           1.53901227       -0.99109534       -0.10006640

H          -1.04502224        0.36926094        0.92580571

H          -1.00793022       -1.02422631       -0.18751680

H          -0.98788058        0.63623225       -0.83954895

H           1.30583447        1.00225372        0.19383714
